In [1]:
# import relevant libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import xgboost as xgb
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
from src.train import build_processor, DATA_PATH, SEED

In [ ]:
# read the dataset and split into training and test sets
loans = pd.read_csv(DATA_PATH)
X = loans.drop(columns="loan_status")
y = loans["loan_status"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=SEED)
X_train.head()

In [ ]:
preprocessor = build_processor(X)